## Data Preparation Steps 

In [1]:
import sqlite3 as sql
import numpy as np 
from sklearn.preprocessing import normalize
crDatabase = sql.connect('customerReviews.db')
cursorReviews = crDatabase.execute("select a.closenessCentrality,a.betweennessCentrality,a.degreeCentrality, b.avgRating, b.totalReviews, b.price, b.unlockedCellRank from productsCentrality a inner join productRanks b on a.product = b.product and a.reviewsDate = b.reviewsDate and a.product in ('B01D0JV7AO','B07GS1V652','B01F482BTK','B01CR1FQMG','B0786PRKBP','B0788ZD69Z') and a.reviewsDate not in ('052319', '053019', '060319', '062019', '062119') order by a.reviewsDate;")
rows=cursorReviews.fetchall()
dataSamples = np.array(rows)

dataSamples1Thru4 = dataSamples[:, :4]
dataSamples5 = dataSamples[:, 4:5]
dataSamples6 = dataSamples[:, 5:6]
dataSamples7 = dataSamples[:, 6:7]

# The below code normalizes the numpy data 
mean = dataSamples5[:].mean(axis=0)
dataSamples5 -= mean 
std = dataSamples5[:].std(axis=0)
dataSamples5 /= std
mean = dataSamples6[:].mean(axis=0)
dataSamples6 -= mean 
std = dataSamples6[:].std(axis=0)
dataSamples6 /= std
float_data = np.concatenate((dataSamples1Thru4, dataSamples5,dataSamples6,dataSamples7), axis=1)
# updated_float_data = series_to_supervised(float_data)
print(float_data.shape)

(438, 7)


In [2]:
# Predict the pollution for the next hour based on the past sequence data
# Based on the LSTM supervised model 
from pandas import DataFrame
from pandas import concat

def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    """
    Frame a time series as a supervised learning dataset.
    Arguments:
        data: Sequence of observations as a list or NumPy array.
        n_in: Number of lag observations as input (X).
        n_out: Number of observations as output (y).
        dropnan: Boolean whether or not to drop rows with NaN values.
    Returns:
        Pandas DataFrame of series framed for supervised learning.
    """
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

## Multivariate LSTM Forecast Model - Normalizing the data

In [3]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder

values = float_data
# integer encode direction
encoder = LabelEncoder()
values[:,6] = encoder.fit_transform(values[:,6])
# ensure all data is float
values = values.astype('float32')
# normalize features
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)
scaled_reverse = scaler.inverse_transform(values)
print(scaled_reverse)
# frame as supervised learning
reframed = series_to_supervised(scaled, 1, 1)
print(reframed.columns.values)
# drop columns we don't want to predict
reframed.drop(['var1(t)', 'var2(t)', 'var3(t)', 'var4(t)', 'var5(t)', 'var6(t)'], axis=1, inplace=True)
print(reframed.columns.values)
print(reframed.head())

[[ 6.8081118e-02  8.3840408e+00  1.6318848e+01 ... -2.7536867e+00
  -2.5457678e+00  1.4960000e+03]
 [ 8.1434108e-02  0.0000000e+00  1.1427933e+01 ... -1.4286635e+00
  -2.4336376e+00  5.1920000e+03]
 [ 1.1330588e-01  1.8695000e+02  1.7356615e+01 ... -6.0070920e+00
  -2.9305222e+00  7.5680000e+03]
 ...
 [ 8.5253775e-02  3.3994231e+00  1.5900667e+01 ... -1.3321662e-01
  -3.7586632e+00  3.3440000e+03]
 [ 1.1999633e-01  2.8790861e+04  2.0367315e+01 ... -5.5575309e+00
  -2.9305222e+00  3.4320000e+03]
 [ 1.1999633e-01  4.8681271e-01  2.9653547e+00 ...  5.7761502e+00
   5.5165157e+00  4.4000000e+02]]
['var1(t-1)' 'var2(t-1)' 'var3(t-1)' 'var4(t-1)' 'var5(t-1)' 'var6(t-1)'
 'var7(t-1)' 'var1(t)' 'var2(t)' 'var3(t)' 'var4(t)' 'var5(t)' 'var6(t)'
 'var7(t)']
['var1(t-1)' 'var2(t-1)' 'var3(t-1)' 'var4(t-1)' 'var5(t-1)' 'var6(t-1)'
 'var7(t-1)' 'var7(t)']
   var1(t-1)  var2(t-1)  var3(t-1)  var4(t-1)  var5(t-1)  var6(t-1)  \
1   0.219660   0.000032   0.307614      0.250   0.231970   0.133846   
2  

## LSTM expects the data in the format [samples, timesteps, features]

In [4]:
# split into train and test sets
values = reframed.values
n_days = 73 * 6
n_train_days = 50 * 6
train = values[:n_train_days, :]
test = values[n_train_days:n_days, :]
# split into input and outputs
train_X, train_y = train[:, :-1], train[:, -1]
test_X, test_y = test[:, :-1], test[:, -1]
# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))
print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)

(300, 1, 7) (300,) (137, 1, 7) (137,)


## Define and Fit Model

In [5]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from matplotlib import pyplot
# design network
model = Sequential()
model.add(LSTM(50, input_shape=(train_X.shape[1], train_X.shape[2])))
model.add(Dense(1))
model.compile(loss='mae', optimizer='adam')
# fit network
history = model.fit(train_X, train_y, epochs=50, batch_size=72, validation_data=(test_X, test_y), verbose=2, shuffle=False)
# plot history
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='test')
pyplot.legend()
pyplot.show()

Using TensorFlow backend.
c:\users\vmt907\documents\mynncode\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\vmt907\documents\mynncode\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\vmt907\documents\mynncode\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\users\vmt907\documents\mynncode\lib\site-packag





Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 300 samples, validate on 137 samples
Epoch 1/50
 - 5s - loss: 0.3612 - val_loss: 0.2809
Epoch 2/50
 - 0s - loss: 0.3263 - val_loss: 0.2542
Epoch 3/50
 - 0s - loss: 0.3006 - val_loss: 0.2360
Epoch 4/50
 - 0s - loss: 0.2797 - val_loss: 0.2239
Epoch 5/50
 - 0s - loss: 0.2628 - val_loss: 0.2167
Epoch 6/50
 - 0s - loss: 0.2489 - val_loss: 0.2112
Epoch 7/50
 - 0s - loss: 0.2382 - val_loss: 0.2085
Epoch 8/50
 - 0s - loss: 0.2310 - val_loss: 0.2089
Epoch 9/50
 - 0s - loss: 0.2259 - val_loss: 0.2114
Epoch 10/50
 - 0s - loss: 0.2225 - val_loss: 0.2142
Epoch 11/50
 - 0s - loss: 0.2205 - val_loss: 0.2166
Epoch 12/50
 - 0s - loss: 0.2191 - val_loss: 0.2188
Epoch 13/50
 - 0s - loss: 0.2180 - val_loss: 0.2208
Epoch 14/50
 - 0s - loss: 0.2170 - val_loss: 0.2221
Epoch 15/50
 - 0s - loss: 0.2163 - val_loss: 0.2228
Epoch 16/50
 - 0s - loss: 0.2158 - val_loss: 0.2231
Epoch 17/50
 - 0s - loss: 0.215

<Figure size 640x480 with 1 Axes>

In [6]:
from numpy import concatenate 
from pandas import concat
from math import sqrt 
from sklearn.metrics import mean_squared_error
# make a prediction
yhat = model.predict(test_X)
test_X = test_X.reshape((test_X.shape[0], test_X.shape[2]))
# invert scaling for forecast
inv_yhat = concatenate((yhat, test_X[:, 1:]), axis=1)
inv_yhat = scaler.inverse_transform(inv_yhat)
inv_yhat = inv_yhat[:,0]
# invert scaling for actual
test_y = test_y.reshape((len(test_y), 1))
inv_y = concatenate((test_y, test_X[:, 1:]), axis=1)
inv_y = scaler.inverse_transform(inv_y)
inv_y = inv_y[:,0]
# calculate RMSE
rmse = sqrt(mean_squared_error(inv_y, inv_yhat))
print('Test RMSE: %.3f' % rmse)

Test RMSE: 0.104


In [7]:
print(train_X.shape, test_X.shape, train_y.shape, test_y.shape, reframed.shape, yhat.shape)
train_tranformed_X = train_X.reshape((train_X.shape[0], train_X.shape[2]))
sevencols_dataset = concatenate((train_tranformed_X, test_X), axis=0)
train_tranformed_y = np.reshape(train_y, (-1, 1))
print(train_tranformed_y.shape)
singlecol_dataset = concatenate((train_tranformed_y, yhat), axis=0)
total_dataset = concatenate((sevencols_dataset[:,:6], singlecol_dataset), axis=1)
total_dataset_inverse = scaler.inverse_transform(total_dataset[:,:7])
total_dataset_inverse[300:,6:7], float_data[300:,6:7]

(300, 1, 7) (137, 7) (300,) (137, 1) (437, 8) (137, 1)
(300, 1)


(array([[34.317646 ],
        [16.946754 ],
        [32.35408  ],
        [28.82535  ],
        [36.060997 ],
        [23.003864 ],
        [33.614304 ],
        [30.780886 ],
        [34.080875 ],
        [34.622265 ],
        [15.313161 ],
        [20.754116 ],
        [17.389261 ],
        [37.96326  ],
        [31.418362 ],
        [34.01576  ],
        [30.949524 ],
        [19.619938 ],
        [33.780823 ],
        [29.13812  ],
        [33.190388 ],
        [33.526047 ],
        [15.883192 ],
        [20.038845 ],
        [34.93606  ],
        [20.524055 ],
        [45.041195 ],
        [41.824085 ],
        [36.489407 ],
        [26.509174 ],
        [23.063028 ],
        [35.902073 ],
        [44.86083  ],
        [31.832466 ],
        [25.104837 ],
        [41.063908 ],
        [35.217354 ],
        [39.982944 ],
        [23.900307 ],
        [26.819487 ],
        [31.344608 ],
        [36.482155 ],
        [40.5079   ],
        [40.664516 ],
        [41.762367 ],
        [3